In [1]:
import json
import time
import os
import re

from modules.preprocessing_protocols import preprocess_text
from modules.extract_results import extract_results_to_txt_file, display_results_by_subdistribution,\
                                    display_results_by_named_entity, display_confusion_matrix

from estnltk import Text
from estnltk.taggers import NerTagger
from estnltk.taggers import WordLevelNerTagger
from estnltk.converters import text_to_json
from estnltk.converters import json_to_text
from estnltk.layer_operations import flatten

from estnltk.taggers.estner.ner_trainer import NerTrainer
from estnltk.taggers.estner.model_storage_util import ModelStorageUtil

In [2]:
def find(name, path):
    for root, dirs, files in os.walk(path):
        if name in files:
            return os.path.join(root, name)

These files don't work because the protocols are written in a different language, which the goldstandard didn't recognise, hence have no goldstandard tags.

In [3]:
files_not_working = ['J2rva_Tyri_V22tsa_id22177_1911a.json', \
                     'J2rva_Tyri_V22tsa_id18538_1894a.json', \
                     'J2rva_Tyri_V22tsa_id22155_1911a.json', \
                     'Saare_Kihelkonna_Kotlandi_id18845_1865a.json', \
                     'P2rnu_Halliste_Abja_id257_1844a.json', \
                     'Saare_Kaarma_Loona_id7575_1899a.json', \
                     'J2rva_Tyri_V22tsa_id22266_1913a.json', \
                     'J2rva_Tyri_V22tsa_id22178_1912a.json']

models = ['model_morph_with_lemmas_and_sentences_and_gazzetteer_and_global_features']

In [4]:
files = {}

with open(os.path.join('..', 'data', 'divided_corpus.txt'), 'r', encoding = 'UTF-8') as f:
    txt = f.readlines()

for fileName in txt:
    file, subdistribution = fileName.split(":")
    files[file] = subdistribution.rstrip("\n")

In [5]:
for model_directory in models:
    for subdistribution in [1, 2, 3, 4, 5]:
        training_subdistributions = []
        for y in [1, 2, 3, 4, 5]:
            if y == subdistribution:
                subdistribution_for_testing = y
            else:
                training_subdistributions.append(y)

            # Getting the filenames to be trained on from the files dictionary.
        filenames = {key: value for key, value in files.items() if int(value) in training_subdistributions}

        # Creating training_texts from the aforementioned filenames.
        print("Valmistan ette treenimistekste.")
        start = time.time()
        training_texts = []
        for filename in filenames:
            with open(os.path.join('..', 'data', 'vallakohtufailid-json-flattened', filename), 'r', encoding='UTF-8') as file:
                if filename in files_not_working:
                    continue
                else:
                    training_texts.append(preprocess_text(json_to_text(file.read())))
        print(f"Treenimistekstid ette valmistatud {time.time() - start} sekundiga.")
        # Setting up the trainer and training.
        print("Alustan NerTaggeri treenimist.")
        start = time.time()
        new_model_dir = os.path.join('models', model_directory)
        modelUtil = ModelStorageUtil( new_model_dir )
        nersettings = modelUtil.load_settings()
        trainer = NerTrainer(nersettings)
        trainer.train( training_texts, layer='gold_wordner', model_dir=new_model_dir )
        print(f"NerTagger treenitud {time.time() - start} sekundiga.")

        # Setting up the new trained nertagger and defining layers to be removed later on.
        nertagger = NerTagger(model_dir = new_model_dir)
        removed_layers = ['sentences', 'morph_analysis', 'compound_tokens', 'ner', 'words', 'tokens']

        # Tagging the files using the new nertagger.
        print("\n\nAlustan failide märgendamist.")
        start = time.time()
        testing_files = {key: value for key, value in files.items() if int(value) == subdistribution_for_testing}
        for file in testing_files:
            with open(find(file.replace(".json", ".txt"), os.path.join('..', 'data', 'vallakohtufailid')), 'r', encoding='UTF-8') as f:
                text = f.read()
                if file == "Tartu_V6nnu_Ahja_id3502_1882a.txt":
                    text = text.replace('..', '. .')
                text = preprocess_text(Text(text))
                nertagger.tag(text)
                text.add_layer(flatten(text['ner'], 'flat_ner'))

                for x in removed_layers:
                    text.pop_layer(x)

                text_to_json(text, file=os.path.join(new_model_dir, 'vallakohtufailid-trained-nertagger', file))
                print(f'Märgendatud fail {file}')
        print(f"Failid märgendatud {time.time() - start} sekundiga.")
    extract_results_to_txt_file(model_directory, files)
print("Programm on lõpetanud oma töö.")

Valmistan ette treenimistekste.
Treenimistekstid ette valmistatud 111.63782596588135 sekundiga.
Alustan NerTaggeri treenimist.
(!) Warning: Location of the new "settings.py" is the same one as the old one. Model's settings are not copied.
Feature generation
type: CRF1d
feature.minfreq: 0.000000
feature.possible_states: 0
feature.possible_transitions: 0
0....1....2....3....4....5....6....7....8....9....10
Number of features: 33519
Seconds required: 0.202

Stochastic Gradient Descent (SGD)
c2: 0.001000
max_iterations: 1000
period: 10
delta: 0.000001

Calibrating the learning rate (eta)
calibration.eta: 0.100000
calibration.rate: 2.000000
calibration.samples: 1000
calibration.candidates: 10
calibration.max_trials: 20
Initial loss: 30999.990087
Trial #1 (eta = 0.100000): 4005.012205
Trial #2 (eta = 0.200000): 5768.592252
Trial #3 (eta = 0.400000): 11186.179400
Trial #4 (eta = 0.800000): 17737.152484
Trial #5 (eta = 1.600000): 39032.107733 (worse)
Trial #6 (eta = 0.050000): 3618.025444
Tria

***** Epoch #36 *****
Loss: 3833.706834
Improvement ratio: 0.286928
Feature L2-norm: 133.578361
Learning rate (eta): 0.049821
Total number of feature updates: 584748
Seconds required for this iteration: 0.209

***** Epoch #37 *****
Loss: 3823.013594
Improvement ratio: 0.222921
Feature L2-norm: 134.839625
Learning rate (eta): 0.049816
Total number of feature updates: 600991
Seconds required for this iteration: 0.207

***** Epoch #38 *****
Loss: 3720.399308
Improvement ratio: 0.166895
Feature L2-norm: 136.066321
Learning rate (eta): 0.049811
Total number of feature updates: 617234
Seconds required for this iteration: 0.202

***** Epoch #39 *****
Loss: 3748.227508
Improvement ratio: 0.132509
Feature L2-norm: 137.292551
Learning rate (eta): 0.049806
Total number of feature updates: 633477
Seconds required for this iteration: 0.212

***** Epoch #40 *****
Loss: 3815.368706
Improvement ratio: 0.088184
Feature L2-norm: 138.589741
Learning rate (eta): 0.049801
Total number of feature updates: 6

Märgendatud fail Tartu_V6nnu_Ahja_id17204_1884a.json
Märgendatud fail Tartu_Torma_Avinurme_id20727_1871a.json
Märgendatud fail L22ne_Vormsi_Vormsi_id24539_1888a.json
Märgendatud fail Tartu_R6ngu_Aakre_id14645_1829a.json
Märgendatud fail Tartu_V6nnu_Ahja_id19396_1887a.json
Märgendatud fail Tartu_Laiuse_Kivij2rve_id6107_1864a.json
Märgendatud fail Tartu_Kodavere_Ranna_id14144_1855a.json
Märgendatud fail J2rva_Tyri_V22tsa_id17503_1888a.json
Märgendatud fail Harju_Hageri_Kohila_id4769_1886a.json
Märgendatud fail J2rva_Tyri_Kirna_id24911_1881a.json
Märgendatud fail Tartu_V6nnu_Ahja_id20077_1888a.json
Märgendatud fail Tartu_V6nnu_Ahja_id9677_1871a.json
Märgendatud fail Tartu_Otep22_Pyhaj2rve_id1315_1884a.json
Märgendatud fail Tartu_Kodavere_Alatskivi_id1603_1878a.json
Märgendatud fail J2rva_Tyri_S2revere_id12869_1879a.json
Märgendatud fail Tartu_Otep22_Pyhaj2rve_id3496_1884a.json
Märgendatud fail P2rnu_Tori_Sindi_id20059_1836a.json
Märgendatud fail L22ne_Reigi_K6rgessaare_id23007_1894a.json


Märgendatud fail V6ru_P6lva_Kiuma_id1633_1880a.json
Märgendatud fail V6ru_Vastseliina_Misso_id25270_1886a.json
Märgendatud fail V6ru_Vastseliina_Misso_id10250_1886a.json
Märgendatud fail Viljandi_Suure-Jaani_Syrgavere_id20998_1877a.json
Märgendatud fail J2rva_Tyri_V22tsa_id16934_1886a.json
Märgendatud fail J2rva_Ambla_Uudekyla_id13485_1867a.json
Märgendatud fail V6ru_P6lva_Peri_id10138_1891a.json
Märgendatud fail Viljandi_Kolga-Jaani_Paenasti_id330_1874a.json
Märgendatud fail Tartu_Torma_Avinurme_id17385_1871a.json
Märgendatud fail V6ru_Vastseliina_Misso_id18830_1882a.json
Märgendatud fail Tartu_Torma_Avinurme_id4086_1858a.json
Märgendatud fail J2rva_Tyri_S2revere_id10947_1871a.json
Märgendatud fail Harju_Kose_Habaja_id735_1874a.json
Märgendatud fail Viru_Haljala_Vihula_id10570_1867a.json
Märgendatud fail Harju_Kose_Triigi_id9690_1869a.json
Märgendatud fail Tartu_Kodavere_Alatskivi_id10323_1880a.json
Märgendatud fail V6ru_Vastseliina_Misso_id21200_1882a.json
Märgendatud fail J2rva_Tyri

***** Epoch #20 *****
Loss: 5087.077559
Improvement ratio: 0.401424
Feature L2-norm: 106.883260
Learning rate (eta): 0.049900
Total number of feature updates: 324020
Seconds required for this iteration: 0.206

***** Epoch #21 *****
Loss: 5019.613275
Improvement ratio: 0.348003
Feature L2-norm: 108.844648
Learning rate (eta): 0.049895
Total number of feature updates: 340221
Seconds required for this iteration: 0.205

***** Epoch #22 *****
Loss: 4918.384905
Improvement ratio: 0.322084
Feature L2-norm: 110.716861
Learning rate (eta): 0.049890
Total number of feature updates: 356422
Seconds required for this iteration: 0.210

***** Epoch #23 *****
Loss: 5093.830990
Improvement ratio: 0.240180
Feature L2-norm: 112.656906
Learning rate (eta): 0.049885
Total number of feature updates: 372623
Seconds required for this iteration: 0.205

***** Epoch #24 *****
Loss: 4674.974126
Improvement ratio: 0.293795
Feature L2-norm: 114.404345
Learning rate (eta): 0.049880
Total number of feature updates: 3

...

Märgendatud fail L22ne_Pyhalepa_K2rdla_id22767_1869a.json
Märgendatud fail Tartu_Kodavere_Alatskivi_id4069_1879a.json
Märgendatud fail J2rva_Tyri_V22tsa_id16522_1885a.json
Märgendatud fail J2rva_Ambla_Ambla_id6971_1884a.json
Märgendatud fail L22ne_Pyhalepa_K2rdla_id25541_1878a.json
Märgendatud fail Harju_Kose_Palvere_id25472_1873a.json
Märgendatud fail V6ru_Vastseliina_Misso_id20265_1882a.json
Märgendatud fail J2rva_Tyri_V22tsa_id18003_1888a.json
Märgendatud fail Tartu_Kodavere_Pala_id20447_1867a.json
Märgendatud fail V6ru_Vastseliina_Misso_id13641_1886a.json
Märgendatud fail Harju_Keila_Keila_id13005_1890a.json
Märgendatud fail V6ru_R2pina_Kahkva_id7771_1888a.json
Märgendatud fail Tartu_N6o_Pangodi_id5095_1889a.json
Märgendatud fail Tartu_N6o_Pangodi_id4146_1889a.json
Märgendatud fail Tartu_Kodavere_Ranna_id14405_1860a.json
Märgendatud fail J2rva_Peetri_V2ike-Kareda_id19122_1868a.json
Märgendatud fail Tartu_V6nnu_Ahja_id19084_1887a.json
Märgendatud fail Harju_Hageri_Kohila_id4010_1890


Stochastic Gradient Descent (SGD)
c2: 0.001000
max_iterations: 1000
period: 10
delta: 0.000001

Calibrating the learning rate (eta)
calibration.eta: 0.100000
calibration.rate: 2.000000
calibration.samples: 1000
calibration.candidates: 10
calibration.max_trials: 20
Initial loss: 33340.335873
Trial #1 (eta = 0.100000): 3252.657389
Trial #2 (eta = 0.200000): 4075.224589
Trial #3 (eta = 0.400000): 7008.205894
Trial #4 (eta = 0.800000): 14321.063549
Trial #5 (eta = 1.600000): 38016.049007 (worse)
Trial #6 (eta = 0.050000): 3248.115033
Trial #7 (eta = 0.025000): 3573.545016
Trial #8 (eta = 0.012500): 4065.131162
Trial #9 (eta = 0.006250): 4727.085952
Trial #10 (eta = 0.003125): 5666.134639
Trial #11 (eta = 0.001563): 7125.606143
Trial #12 (eta = 0.000781): 9485.662622
Trial #13 (eta = 0.000391): 13041.862592
Trial #14 (eta = 0.000195): 17832.568566
Trial #15 (eta = 0.000098): 23542.264904
Best learning rate (eta): 0.050000
Seconds required: 0.237

***** Epoch #1 *****
Loss: 24362.466215
Fea

***** Epoch #37 *****
Loss: 3746.814917
Improvement ratio: 0.163950
Feature L2-norm: 133.853838
Learning rate (eta): 0.049816
Total number of feature updates: 577718
Seconds required for this iteration: 0.236

***** Epoch #38 *****
Loss: 3679.071192
Improvement ratio: 0.163792
Feature L2-norm: 135.081868
Learning rate (eta): 0.049811
Total number of feature updates: 593332
Seconds required for this iteration: 0.228

***** Epoch #39 *****
Loss: 3672.424746
Improvement ratio: 0.141551
Feature L2-norm: 136.311794
Learning rate (eta): 0.049806
Total number of feature updates: 608946
Seconds required for this iteration: 0.230

***** Epoch #40 *****
Loss: 3612.004020
Improvement ratio: 0.145538
Feature L2-norm: 137.506899
Learning rate (eta): 0.049801
Total number of feature updates: 624560
Seconds required for this iteration: 0.230

***** Epoch #41 *****
Loss: 3608.955926
Improvement ratio: 0.126427
Feature L2-norm: 138.681386
Learning rate (eta): 0.049796
Total number of feature updates: 6

Märgendatud fail Harju_Kuusalu_Kolga_id11902_1888a.json
Märgendatud fail J2rva_Peetri_V2ike-Kareda_id22150_1879a.json
Märgendatud fail Tartu_Kodavere_Pala_id22870_1872a.json
Märgendatud fail Tartu_V6nnu_Ahja_id13565_1881a.json
Märgendatud fail Harju_Juuru_Juuru_id17866_1869a.json
Märgendatud fail Harju_Juuru_Kaiu_id931_1912a.json
Märgendatud fail J2rva_Anna_Eivere_id6607_1880a.json
Märgendatud fail L22ne_Kullamaa_Piirsalu_id16723_1888a.json
Märgendatud fail J2rva_Tyri_V22tsa_id17604_1888a.json
Märgendatud fail Tartu_V6nnu_Ahja_id14708_1882a.json
Märgendatud fail Tartu_Kodavere_Alatskivi_id9914_1880a.json
Märgendatud fail L22ne_Pyhalepa_K2rdla_id23461_1873a.json
Märgendatud fail Tartu_V6nnu_Ahja_id21842_1868a.json
Märgendatud fail Harju_J6el2htme_J6el2htme_id7633_1870a.json
Märgendatud fail Tartu_Kodavere_Alatskivi_id13040_1856a.json
Märgendatud fail Saare_P8ide_Laimjala_id6577_1917a.json
Märgendatud fail J2rva_Tyri_Kirna_id24066_1879a.json
Märgendatud fail L22ne_Ridala_Sinalepa_id25489

Märgendatud fail Tartu_Otep22_Pyhaj2rve_id11749_1885a.json
Märgendatud fail Harju_Harju-Madise_Padise-Kloostri_id3283_1885a.json
Märgendatud fail Tartu_V6nnu_Ahja_id15058_1883a.json
Märgendatud fail Tartu_Torma_Avinurme_id14255_1903a.json
Märgendatud fail Tartu_Kodavere_Ranna_id15105_1863a.json
Märgendatud fail Harju_Hageri_Kohila_id10294_1868a.json
Märgendatud fail Tartu_Kodavere_Pala_id16727_1855a.json
Märgendatud fail J2rva_Peetri_V2ike-Kareda_id22431_1880a.json
Märgendatud fail Saare_Kihelkonna_Kotlandi_id21549_1869a.json
Märgendatud fail J2rva_Tyri_V22tsa_id18782_1897a.json
Märgendatud fail J2rva_Tyri_V22tsa_id16955_1887a.json
Märgendatud fail J2rva_J2rva-Jaani_Einmanni_id9007_1868a.json
Märgendatud fail Tartu_V6nnu_Ahja_id23568_1895a.json
Märgendatud fail Tartu_Kodavere_Alatskivi_id1504_1865a.json
Märgendatud fail P2rnu_P2rnu-Elisabethi_Sauga_id18104_1877a.json
Märgendatud fail L22ne_Emmaste_Emmaste_id8296_1894a.json
Märgendatud fail Tartu_V6nnu_Ahja_id13380_1881a.json
Märgendatu